In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

import scipy.sparse as sparse

%matplotlib inline

from implicit.als import AlternatingLeastSquares

#### посмотрим на каким сайтам какие тэги наиболее близки

In [132]:
# какому сайту какие тэги сколько раз поставили
book_tags = pd.read_csv('hetrec2011-delicious-2k/bookmark_tags.dat', sep='\t')
# описание сайтов
bookmarks = pd.read_csv('hetrec2011-delicious-2k/bookmarks_utf8.dat', encoding='utf_8', sep='\t')
# описание тэгов
tags = pd.read_csv('hetrec2011-delicious-2k/tags_utf8.dat', encoding='utf_8', sep='\t')


In [133]:
book_tags.head()

,bookmarkID,tagID,tagWeight
0,1,2,276
1,1,206,48
2,1,208,165
3,1,272,55
4,1,459,67


In [134]:
bookmarks.head()

,id,md5,title,url,md5Principal,urlPrincipal
0,1,ab4954b633ddaf5b5bba6e9b71aa6b70,IFLA - The official website of the Internation...,http://www.ifla.org/,7f431306c428457bc4e12b15634484f,www.ifla.org
1,2,2221e9cd106d269dd34682666f576fa3,gcdp-e.pdf (application/pdf Object),http://archive.ifla.org/VII/s14/nd1/gcdp-e.pdf,1ef8cfcfe968101fa9b4e301847503d4,archive.ifla.org
2,7,c97c571dadaddbbb493126a0d4d01ba3,EdSelect,http://www.edselect.com/,792fd7eb20143386d0c4eb193c6124d,www.edselect.com
3,8,25bfe8dca0ef263ec9c341b9f16c38b5,Cool Canada (Collections Canada),http://www.collectionscanada.gc.ca/cool/index-...,6fce4f6391516f0732531d9cfacda5b7,www.collectionscanada.gc.ca
4,9,c97284629e17b8e2861afaacd59918bc,Kidsreads.com,http://www.kidsreads.com/,5854ce8404857a45373eea01a3d98000,www.kidsreads.com


In [135]:
tags.head()

,id,value
0,1,collection_development
1,2,library
2,3,collection
3,4,development
4,5,lesson_plan


In [140]:
data = book_tags.merge(bookmarks[['urlPrincipal','id']], how='left', left_on='bookmarkID', right_on='id')
data = data.merge(tags[['value','id']], how='left', left_on='tagID', right_on='id')
data = data.drop(['bookmarkID','tagID','id_x', 'id_y'], axis=1)
data = data.dropna()
data.head()

,tagWeight,urlPrincipal,value
0,276,www.ifla.org,library
1,48,www.ifla.org,reference
2,165,www.ifla.org,international
3,55,www.ifla.org,research
4,67,www.ifla.org,information


In [141]:
data.describe()

,tagWeight
count,485993.000000
mean,126.627295
std,526.828563
min,2.000000
25%,4.000000
50%,14.000000
75%,63.000000
max,41511.000000


In [142]:
data['bookmarkID_id'] = data['urlPrincipal'].astype("category").cat.codes
data['tagID_id'] = data['value'].astype("category").cat.codes

In [143]:
data.head()

,tagWeight,urlPrincipal,value,bookmarkID_id,tagID_id
0,276,www.ifla.org,library,25935,7027
1,48,www.ifla.org,reference,25935,10362
2,165,www.ifla.org,international,25935,6213
3,55,www.ifla.org,research,25935,10472
4,67,www.ifla.org,information,25935,6078


In [148]:
tags_lookup = data[['tagID_id', 'value']].drop_duplicates()
tags_lookup['tagID_id'] = tags_lookup.tagID_id.astype(str)
bookm_lookup = data[['bookmarkID_id', 'urlPrincipal']].drop_duplicates()
bookm_lookup['bookmarkID_id'] = bookm_lookup.bookmarkID_id.astype(str)

In [150]:
display(tags_lookup.head(), bookm_lookup.head())

,tagID_id,value
0,7027,library
1,10362,reference
2,6213,international
3,10472,research
4,6078,information


,bookmarkID_id,urlPrincipal
0,25935,www.ifla.org
10,911,archive.ifla.org
19,22937,www.edselect.com
29,21061,www.collectionscanada.gc.ca
30,27210,www.kidsreads.com


In [152]:
tags_id_name = {}

for index, row in tqdm_notebook(tags_lookup.iterrows()):
    tags_id_name[row.tagID_id] = row.value

bookm_id_name = {}

for index, row in tqdm_notebook(bookm_lookup.iterrows()):
    bookm_id_name[row.bookmarkID_id] = row.urlPrincipal

In [155]:
tags_id_name, bookm_id_name

({'7027': 'library',
  '10362': 'reference',
  '6213': 'international',
  '10472': 'research',
  '6078': 'information',
  '7026': 'libraries',
  '801': 'association',
  '802': 'associations',
  '8872': 'organizations',
  '5939': 'ifla',
  '2324': 'collection_development',
  '2323': 'collection',
  '3234': 'development',
  '5422': 'guidelines',
  '9562': 'policies',
  '2325': 'collectiondevelopment',
  '7140': 'literacy',
  '3796': 'education',
  '6912': 'learning',
  '12380': 'technology',
  '10493': 'resources',
  '7114': 'links',
  '6991': 'lessons',
  '6990': 'lessonplans',
  '2903': 'curriculum',
  '1790': 'canada',
  '10256': 'reading',
  '6697': 'kids',
  '1471': 'books',
  '10539': 'reviews',
  '919': 'authors',
  '7146': 'literature',
  '2057': 'children',
  '1461': 'booklists',
  '1531': 'boys',
  '7999': 'moodle',
  '2314': 'collaboration',
  '12349': 'teaching',
  '12701': 'tools',
  '11609': 'software',
  '3859': 'elearning',
  '3888': 'elementary',
  '10833': 'san',
  '118

In [156]:
data

,tagWeight,urlPrincipal,value,bookmarkID_id,tagID_id
0,276,www.ifla.org,library,25935,7027
1,48,www.ifla.org,reference,25935,10362
2,165,www.ifla.org,international,25935,6213
3,55,www.ifla.org,research,25935,10472
4,67,www.ifla.org,information,25935,6078
5,181,www.ifla.org,libraries,25935,7026
6,108,www.ifla.org,association,25935,801
7,150,www.ifla.org,associations,25935,802
8,62,www.ifla.org,organizations,25935,8872
9,270,www.ifla.org,ifla,25935,5939


In [157]:
data = data.drop(['urlPrincipal', 'value'], axis=1)

In [158]:
len(data)

485993

In [159]:
data.describe()

,tagWeight,bookmarkID_id,tagID_id
count,485993.000000,485993.000000,485993.000000
mean,126.627295,19249.858451,7464.257288
std,526.828563,11235.397294,4183.642536
min,2.000000,0.000000,0.000000
25%,4.000000,9293.000000,3796.000000
50%,14.000000,19124.000000,7322.000000
75%,63.000000,29355.000000,11377.000000
max,41511.000000,38188.000000,14203.000000


In [160]:
bookmarks = list(np.sort(data.bookmarkID_id.unique()))
tags = list(np.sort(data.tagID_id.unique()))
counts = list(data.tagWeight)

In [161]:
bookmarks[:5]

[0, 1, 2, 3, 4]

In [162]:
tags[:5]

[0, 1, 2, 3, 4]

In [163]:
counts[:5]

[276, 48, 165, 55, 67]

In [164]:
len(bookmarks)

38189

In [165]:
len(tags)

14204

In [166]:
rows = data.bookmarkID_id.astype(int)
cols = data.tagID_id.astype(int)

In [170]:
data_sparse = sparse.csr_matrix((counts, (cols, rows)), shape=(len(tags), len(bookmarks)))

In [171]:
model = AlternatingLeastSquares(factors=50)
model.fit(data_sparse)

100%|██████████| 15.0/15 [00:01<00:00, 12.12it/s]


In [194]:
bookm_id_name['25935']

'www.ifla.org'

In [196]:
userid = 25935

user_items = data_sparse.T.tocsr()
recommendations = model.recommend(userid, user_items)

In [197]:
recommendations

[(9840, 0.85303366),
 (189, 0.76723564),
 (5586, 0.74225426),
 (10493, 0.6723623),
 (7024, 0.668072),
 (13971, 0.65591717),
 (8396, 0.6508523),
 (10923, 0.6326566),
 (4032, 0.6291611),
 (11896, 0.608675)]

In [198]:
for r in recommendations:
    print(tags_id_name[str(r[0])])

professional
academic
health
resources
librarians
world
news
school
environment
statistics


In [209]:
itemid = 5586
related = model.similar_items(itemid)

In [200]:
related

[(5586, 0.060699537),
 (8588, 0.053688116),
 (7635, 0.053524148),
 (3289, 0.05323045),
 (7639, 0.05281232),
 (4539, 0.05237453),
 (4196, 0.051610768),
 (5588, 0.051359188),
 (3741, 0.050907888),
 (379, 0.050348084)]

In [211]:
tags_id_name[str(itemid)]

'health'

In [204]:
for a in related:
    print(tags_id_name[str(a[0])])

health
nutrition
medical
diet
medicine
fitness
exercise
healthcare
ecology
aging
